<a href="https://colab.research.google.com/github/MartsenkoVS/Kyokushinkai_audio_classification/blob/main/Audio_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт библиотек

In [ ]:
%%capture
!pip install sounddevice
!sudo apt-get install libportaudio2

In [ ]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
import pickle
import sounddevice as sd
import soundfile as sf
from IPython.display import Audio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Обработка аудиодорожки для передачи в модель:

In [ ]:
labels_list = ['hadgime', 'kato1', 'kato2', 'kato3', 'kato4', 'kiai', 'naure', 'oss']

In [ ]:
# Загрузка обученной модели
model = load_model('/content/drive/MyDrive/Стажировка "Кёкусинкай"/models/best_model_librosa.h5')

# Загрузка параметров нормализации
with open('/content/drive/MyDrive/Стажировка "Кёкусинкай"/models/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [ ]:
# Функция для вывода результата предикта модели:
def model_predict(x_test, labels_list, model):
  predict = model.predict(x_test, verbose=0)

  pred_index = np.argmax(predict, axis=1)
  y_pred = labels_list[pred_index[0]]
  predicted_class_confidence = predict[0][pred_index[0]]

  return y_pred, predicted_class_confidence

In [ ]:
# Извлечение признаков из аудио
def extract_features_librosa(normalized_data, sr, scaler):
  x_test = []

  # Подрезка тишины в аудио
  audio, _ = librosa.effects.trim(normalized_data, top_db=20)

  # Извлечение MFCC
  mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
  mfccs_mean = np.mean(mfccs, axis=1)

  # Извлечение Zero Crossing Rate
  zcr = librosa.feature.zero_crossing_rate(audio)
  zcr_mean = np.mean(zcr)

  # Извлечение спектральной ширины
  spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
  spectral_bandwidth_mean = np.mean(spectral_bandwidth)

  # Извлечение спектрального центроида
  spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
  spectral_centroid_mean = np.mean(spectral_centroid)

  # Извлечение RMS
  rms = librosa.feature.rms(y=audio)
  rms_mean = np.mean(rms)

  # Собираем все признаки в один массив
  features = np.hstack((mfccs_mean, zcr_mean, spectral_bandwidth_mean, spectral_centroid_mean, rms_mean))

  x_test.append(features)
  x_test = np.array(x_test)
  # Нормализация признаков
  x_test_scaled = scaler.transform(x_test)

  return x_test_scaled

In [ ]:
# Общая функция для обработки аудио в виде аудиопотока
def predict_from_audio_stream(model, labels_list, scaler, label_mode=3):
    # режимы определения команд:
    if label_mode == 1:
        valid_labels = ['hadgime']
    elif label_mode == 2:
        valid_labels = ['hadgime', 'naure']
    elif label_mode == 3:
        valid_labels = labels_list  # Все команды

    # Словарь с порогами уверенности для каждой команды
    confidence_thresholds = {
        'hadgime': 0.9,
        'kiai': 0.9,
        'kato1': 0.5,
        'kato2': 0.97,
        'kato3': 0.95,
        'kato4': 0.89,
        'naure': 0.92,
        'oss': 0.89
    }

    RATE = 22050  # Целевая частота дискретизации, на которой обучалась модель
    CHUNK_DURATION = 0.25  # длительность кусочка в секундах
    BUFFER_DURATION = 1  # длительность буфера в секундах
    BUFFER_SIZE = int(RATE * BUFFER_DURATION)  # размер буфера в сэмплах
    CHUNK_SIZE = int(RATE * CHUNK_DURATION)  # размер кусочка в сэмплах
    TIME_TO_STOP_STREAM = 36 # время для проверки команды naure, для остановки аудиопотока

    # Список для записи предсказаний
    predict_list = []
    predict_data = {'timestamp stop': 0, 'predict': 0}
    predict_list.append(predict_data)

    # Создание пустого буфера
    buffer = np.zeros(BUFFER_SIZE, dtype=np.float32)
    total_samples = 0 # счетчик обработанных сэмплов

    # функция обработки аудиопотока
    def callback(indata, frames, time, status):
        nonlocal total_samples
        # вывод информации об ошибке аудиопотока
        if status:
            print(f"Error: {status}")

        # Обновление буфера
        buffer[:-frames] = buffer[frames:]
        buffer[-frames:] = indata

        total_samples += frames # Общее кол-во обработанных семплов

        if total_samples >= BUFFER_SIZE:  # Начать обработку после заполнения буфера

          # Извлекаем аудио признаки через librosa
          x_test_scaled = extract_features_librosa(buffer, RATE, scaler)

          # Получаем предсказание модели
          y_pred, predicted_class_confidence = model_predict(x_test_scaled, labels_list, model)
          if y_pred in valid_labels and predicted_class_confidence > confidence_thresholds[y_pred]:

            # Замеряем метку времени
            timestamp = round((total_samples / RATE), 2)
            timestamp_start = timestamp - BUFFER_DURATION
            # Предполагается, что видео 30 кадров в секунду
            frame = int(round((timestamp * 30), 0))
            frame_start = frame - BUFFER_DURATION * 30

            # Добавляем предикт в список
            if ((y_pred != predict_list[-1]['predict'])
             or ((y_pred == predict_list[-1]['predict']) and (timestamp - predict_list[-1]['timestamp stop'] > 2.5))
             ):
              predict_data = {'timestamp start':timestamp_start,
                              'timestamp stop': timestamp,
                              'frame start': frame_start,
                              'frame stop': frame,
                              'predict': y_pred,
                              'confidence': predicted_class_confidence,
                              }
              predict_list.append(predict_data)

              # Вывод результата
              print(f"Время: {timestamp_start:.2f} - {timestamp:.2f} секунд, Номер кадра: {frame_start} - {frame}, Предсказание: {y_pred}, Confidence: {predicted_class_confidence * 100:.2f}%")

              if ((label_mode != 1) and (timestamp > TIME_TO_STOP_STREAM) and (y_pred == 'naure')):
                print('Прозвучала команда "naure", выступление спортсмена закончено, прекращаю анализ аудио')
                raise sd.CallbackStop

    # Запуск аудиопотока с моноканалом, частотой дискретизации 22050, float32 (для извлечения признаков через librosa)
    with sd.InputStream(callback=callback, channels=1, samplerate=RATE, blocksize=CHUNK_SIZE, dtype='float32'):
        print("Listening, press Ctrl+C to stop.")
        try:
            while True:
                sd.sleep(TIME_TO_STOP_STREAM * 1000)  # Ожидаем 36 секунд или до остановки потока
        except KeyboardInterrupt:
            print("Stream stopped by user.")
        except sd.CallbackStop:
            print("Stream stopped by callback condition.")

    return predict_list[1:]

## Обработка аудио из файла:

In [ ]:
# Общая функция для обработки аудио из файла
def predict_from_audio_librosa(audio_path, model, labels_list, scaler, label_mode=3, save_audio=False):
    # режимы определения команд:
    if label_mode == 1:
        valid_labels = ['hadgime']
    elif label_mode == 2:
        valid_labels = ['hadgime', 'naure']
    elif label_mode == 3:
        valid_labels = labels_list  # Все команды

    # Словарь с порогами уверенности для каждой команды
    confidence_thresholds = {
        'hadgime': 0.9,
        'kiai': 0.9,
        'kato1': 0.5,
        'kato2': 0.97,
        'kato3': 0.95,
        'kato4': 0.89,
        'naure': 0.9,
        'oss': 0.89
    }

    RATE = 22050  # Целевая частота дискретизации, на которой обучалась модель
    CHUNK_DURATION = 0.25  # длительность кусочка в секундах
    BUFFER_DURATION = 1  # длительность буфера в секундах
    BUFFER_SIZE = int(RATE * BUFFER_DURATION)  # размер буфера в сэмплах
    CHUNK_SIZE = int(RATE * CHUNK_DURATION)  # размер кусочка в сэмплах
    TIME_TO_STOP_STREAM = 36 # время для проверки команды naure, для остановки аудиопотока

    # Список для записи предсказаний
    predict_list = []
    predict_data = {'timestamp stop': 0, 'predict': 0}
    predict_list.append(predict_data)

    # Загрузка аудио файла
    audio, _ = librosa.load(audio_path, sr=RATE)

    # Обработка аудио файла кусочками
    for start in range(0, len(audio) - BUFFER_SIZE + 1, CHUNK_SIZE):
        buffer = audio[start:start + BUFFER_SIZE]

        # Извлекаем аудио признаки через librosa
        x_test_scaled = extract_features_librosa(buffer, RATE, scaler)

        # Получаем предсказание модели
        y_pred, predicted_class_confidence = model_predict(x_test_scaled, labels_list, model)
        if y_pred in valid_labels and predicted_class_confidence > confidence_thresholds[y_pred]:

          # Замеряем метку времени
          timestamp_start = round((start / RATE), 2)
          timestamp_stop = timestamp_start + BUFFER_DURATION
          frame_start = int(round((timestamp_start * 30), 0))
          frame_stop = frame_start + BUFFER_DURATION * 30

          # Добавляем предикт в список
          if ((y_pred != predict_list[-1]['predict'])
           or ((y_pred == predict_list[-1]['predict']) and (timestamp_stop - predict_list[-1]['timestamp stop'] > 2.5))
           ):
            predict_data = {'timestamp start':timestamp_start,
                            'timestamp stop': timestamp_stop,
                            'frame start': frame_start,
                            'frame stop': frame_stop,
                            'predict': y_pred,
                            'confidence': predicted_class_confidence,
                            }
            predict_list.append(predict_data)

            # Вывод результата
            print(f"Время: {timestamp_start:.2f} - {timestamp_stop:.2f} секунд, Номер кадра: {frame_start} - {frame_stop}, Предсказание: {y_pred}, Confidence: {predicted_class_confidence * 100:.2f}%")

            if save_audio == True:
              # Сохранение куска аудио для анализа работы модели
              audio_name = os.path.basename(audio_path)[:-4]
              os.makedirs(f'/content/predicted_audio/{audio_name}', exist_ok=True)
              sf.write(f'/content/predicted_audio/{audio_name}/{timestamp_stop}_{y_pred}.wav', buffer, RATE)

            if ((label_mode != 1) and (timestamp_stop > TIME_TO_STOP_STREAM) and (y_pred == 'naure')):
              print('Прозвучала команда "naure", выступление спортсмена закончено, прекращаю анализ аудио')
              break

    return predict_list[1:]

##Проверим работу модели на 4 аудио от заказчика:

In [ ]:
# Обрабатываем 4 тестовых аудио
test_dir = '/content/drive/MyDrive/Стажировка "Кёкусинкай"/datasets/test'

for audio_name in os.scandir(test_dir):
  if audio_name.is_file() and audio_name.name.endswith('.wav'):
    audio_file_path = os.path.join(test_dir, audio_name)
    timestamps_folder_path = os.path.join(test_dir, 'timestamps')
    timestamps_path = os.path.join(timestamps_folder_path, f'{audio_name.name[:-4]}.txt')
    with open (timestamps_path, 'r') as f:
      timestamps = f.read()
    print(audio_name.name)
    print('Метки:')
    print(timestamps)
    print('Предсказания модели:')
    predict_list = predict_from_audio_librosa(audio_file_path, model, labels_list, scaler, label_mode=3, save_audio=True)
    print()

20240209_181457.wav
Метки:
5.683204	6.392208	oss
9.378961	10.026556	oss
10.601575	11.653243	kato1
16.611398	17.393018	hadgime
24.968415	25.990837	kia
33.747823	34.679451	kia
40.090590	41.189003	naure

Предсказания модели:
Время: 5.00 - 6.00 секунд, Номер кадра: 150 - 180, Предсказание: oss, Confidence: 100.00%
Время: 8.75 - 9.75 секунд, Номер кадра: 262 - 292, Предсказание: oss, Confidence: 100.00%
Время: 10.50 - 11.50 секунд, Номер кадра: 315 - 345, Предсказание: kato1, Confidence: 63.36%
Время: 11.75 - 12.75 секунд, Номер кадра: 352 - 382, Предсказание: kato2, Confidence: 99.13%
Время: 15.75 - 16.75 секунд, Номер кадра: 472 - 502, Предсказание: hadgime, Confidence: 99.97%
Время: 24.25 - 25.25 секунд, Номер кадра: 728 - 758, Предсказание: kiai, Confidence: 100.00%
Время: 33.00 - 34.00 секунд, Номер кадра: 990 - 1020, Предсказание: kiai, Confidence: 100.00%
Время: 39.75 - 40.75 секунд, Номер кадра: 1192 - 1222, Предсказание: naure, Confidence: 92.80%
Прозвучала команда "naure", выступл

In [ ]:
# Пример вывода списка предсказаний
predict_list

[{'timestamp start': 3.25,
  'timestamp stop': 4.25,
  'frame start': 98,
  'frame stop': 128,
  'predict': 'oss',
  'confidence': 0.89838886},
 {'timestamp start': 6.0,
  'timestamp stop': 7.0,
  'frame start': 180,
  'frame stop': 210,
  'predict': 'oss',
  'confidence': 0.9074996},
 {'timestamp start': 8.0,
  'timestamp stop': 9.0,
  'frame start': 240,
  'frame stop': 270,
  'predict': 'kato3',
  'confidence': 0.96803755},
 {'timestamp start': 12.0,
  'timestamp stop': 13.0,
  'frame start': 360,
  'frame stop': 390,
  'predict': 'hadgime',
  'confidence': 0.9893026},
 {'timestamp start': 20.5,
  'timestamp stop': 21.5,
  'frame start': 615,
  'frame stop': 645,
  'predict': 'kiai',
  'confidence': 0.99889994},
 {'timestamp start': 22.25,
  'timestamp stop': 23.25,
  'frame start': 668,
  'frame stop': 698,
  'predict': 'naure',
  'confidence': 0.93915033},
 {'timestamp start': 30.0,
  'timestamp stop': 31.0,
  'frame start': 900,
  'frame stop': 930,
  'predict': 'kiai',
  'confid